# Main

> Module die alle functies die nodig zijn om de snelste route te vinden langs alle peilbuizen gegroepeerd per project.

In [ ]:
#| default_exp main

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import yaml
import pandas as pd
import logging
from datetime import datetime
from fastcore.utils import Path
from tqdm import tqdm

from project.data_get import get_data_from_azuresql, load_pickle
from project.route_get import create_optimized_route
from project.data_export import get_waypoint_coords, convert_routejson_to_df, create_route_prev_next, google_maps_route_url
from project.utils import get_project_root, make_filesystem_friendly, save_route_url, setup_logging

Get the settings from the settings.yaml file.

In [ ]:
#| export
project_root = get_project_root()

with open(project_root / 'settings.yaml', 'r') as f:
    settings = yaml.safe_load(f)

Get the data from the Azure SQL database in a pandas DataFrame or from a pickle file.

In [ ]:
#| export
# Only used for testing
use_pickle=True

if use_pickle:
    peilbuizen_df = load_pickle(settings['files']['pickle_file_input'])
else:
    peilbuizen_df = get_data_from_azuresql(sql_statement=settings['sql_statement']['peilbuizen'], 
                                       save_to_pickle=settings['files']['save_to_pickle'])

df_grouped = peilbuizen_df.groupby('project')
test_df_group = df_grouped.get_group('---')
test_name_group = 'group_name'

df_random_test = peilbuizen_df.sample(n=100)

In [ ]:
#| hide
test_df_group.head()

,Id,Longitude,latitude,latest_measure_date,project
241,27GC901A;408995,6.180674,52.359436,2024-04-03 10:55:02,---
242,27HC007B;410283,6.202980,52.306282,2024-11-04 11:26:39,---
296,21FC011A;394286,6.207653,52.601814,2024-04-09 09:51:57,---
389,B21E0010;416303,6.181055,52.613954,2024-04-09 10:43:36,---
451,B21F0003;416597,6.219551,52.650246,2024-04-09 10:57:31,---


In [ ]:
#| hide
df_random_test.head()

,Id,Longitude,latitude,latest_measure_date,project
588,21FL0008;394678,6.248350,52.644864,2024-10-10 14:04:21,Primair meetnet blok 2
408,PB53-92.7A2;1137057,6.076334,52.537876,2023-04-25 11:02:17,None
984,53-35.7A;940459,6.112688,52.442759,2024-12-12 10:51:55,None
867,17AC612A;1310086,6.442024,52.823646,2024-08-30 09:55:35,None
883,B22C1349;390212,6.403365,52.559865,2024-10-10 10:52:51,Primair meetnet blok 2


For every project in the peilbuizen dataframe, the shortest route is calculated and saved to an Excel file and a URL file.

In [ ]:
#| export

def create_group_route(start_address: str,
                       group_df: pd.DataFrame, 
                       route_profile: str,
                       project_name: str,
                       output_dir: str = 'output',
                       current_date: str = None,
                       ) -> None:
    """
    Create optimized route for a group and save results
    """
    if current_date is None:
        current_date = datetime.now().strftime('%Y-%m-%d')
        
    # Create output directory
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    try:
        # Get optimized route
        route_json = create_optimized_route(start_address, group_df, route_profile)
        route_coords = get_waypoint_coords(route_json)
        
        # Create route dataframe
        optimized_route = convert_routejson_to_df(route_coords, group_df)
        route_table = create_route_prev_next(optimized_route)
        
        # Create filenames
        safe_project_name = make_filesystem_friendly(project_name)
        base_filename = f"peilbuizenroute_{safe_project_name}_{current_date}"
        
        # Save Excel file
        excel_path = output_path / f"{base_filename}.xlsx"
        route_table.to_excel(excel_path)
        
        # Save URL shortcut
        url = google_maps_route_url(route_coords)
        save_route_url(url, output_dir, f"{base_filename}.url")
        
    except Exception as e:
        print(f"Error processing group {project_name}: {str(e)}")

In [ ]:
#| export
create_group_route(start_address=settings['calculation']['startlocation'],
                    group_df=test_df_group,
                    route_profile=settings['calculation']['distance_calculation_method'],
                    project_name=test_name_group,
                    output_dir=settings['files']['path_results'])


In [ ]:
#| export

def process_peilbuizen_routes(df: pd.DataFrame, 
                            start_address: str,
                            route_profile: str,
                            output_dir: str = 'output') -> None:
    """
    Process peilbuizen dataframe, create optimized routes for each group,
    and save results to Excel and URL files
    
    Args:
        df: DataFrame with peilbuizen data
        start_address: Starting location address
        output_dir: Directory to save output files
    """
    # Create output directory and setup logging
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    current_date = datetime.now().strftime('%Y-%m-%d')
    logger = setup_logging(output_dir, current_date)
        
    # Process each group
    grouped = df.groupby('project')
    total_groups = len(grouped)
    
    for project_name, group_df in tqdm(grouped, total=total_groups,desc="Processing peilbuis routes per project."):
        if len(group_df) < 4:
            msg = f"Project {project_name} heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos."
            print(msg)
            logger.info(msg)
        else:
            try:
                create_group_route(
                    start_address=start_address,
                    group_df=group_df,
                    route_profile=route_profile,
                    project_name=project_name,
                    output_dir=output_dir,
                    current_date=current_date
                )
                logger.info(f"Successfully processed route for project: {project_name}")
            except Exception as e:
                error_msg = f"Error processing project {project_name}: {str(e)}"
                print(error_msg)
                logger.error(error_msg)

In [ ]:
#| export

process_peilbuizen_routes(df=peilbuizen_df,
                         start_address=settings['calculation']['startlocation'],
                         route_profile=settings['calculation']['distance_calculation_method'],
                         output_dir=settings['files']['path_results'])

Processing peilbuis routes per project.:   0%|          | 0/16 [00:00<?, ?it/s]2025-01-24 19:24:32:INFO:project.utils:Project Boetelerveld_nw heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:32:INFO:project.utils:Project Holtingerveld _Ootmaanlanden en Koningschut heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:32:INFO:project.utils:Project Mastenbroek-IJssel heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:32:INFO:project.utils:Project Meetnet De Wijk II heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.


Project Boetelerveld_nw heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Holtingerveld _Ootmaanlanden en Koningschut heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Mastenbroek-IJssel heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Meetnet De Wijk II heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-01-24 19:24:32:INFO:project.utils:Successfully processed route for project: Olde Maten en Veerslootlanden
Processing peilbuis routes per project.:  31%|███▏      | 5/16 [00:00<00:00, 11.09it/s]2025-01-24 19:24:32:INFO:project.utils:Project Oldematen Reevediep heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:32:INFO:project.utils:Project Oude Diep Mantinge Bos_Zand heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.


Error processing group Olde Maten en Veerslootlanden: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 6.1215398 52.6136725.'}, 'info': {'engine': {'build_date': '2024-12-02T11:09:21Z', 'graph_version': '1', 'version': '9.0.0'}, 'timestamp': 1737743072378}})
Project Oldematen Reevediep heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Oude Diep Mantinge Bos_Zand heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.


2025-01-24 19:24:33:INFO:project.utils:Successfully processed route for project: Primair meetnet blok 1
Processing peilbuis routes per project.:  50%|█████     | 8/16 [00:01<00:01,  5.09it/s]

Error processing group Primair meetnet blok 1: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 5: 6.2932497 52.3568122.'}, 'info': {'engine': {'build_date': '2024-12-02T11:09:21Z', 'graph_version': '1', 'version': '9.0.0'}, 'timestamp': 1737743073339}})


2025-01-24 19:24:34:INFO:project.utils:Successfully processed route for project: Primair meetnet blok 2
Processing peilbuis routes per project.:  62%|██████▎   | 10/16 [00:03<00:02,  2.08it/s]2025-01-24 19:24:35:INFO:project.utils:Project Reevediep heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:35:INFO:project.utils:Project Vecht heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:35:INFO:project.utils:Project Vechterweerd heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:35:INFO:project.utils:Project Wabos-KRW heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:35:INFO:project.utils:Project peilbesluit Kostverlorenzijl en Kloosterzijl heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
2025-01-24 19:24:35:INFO:project.utils:Project zandwinplassen heeft minder dan 3 of minder locaties. Opt

Project Reevediep heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Vecht heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Vechterweerd heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project Wabos-KRW heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project peilbesluit Kostverlorenzijl en Kloosterzijl heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.
Project zandwinplassen heeft minder dan 3 of minder locaties. Optimale route uitrekenen is zinloos.


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()